In [2]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import TensorBoard
from keras.callbacks import ModelCheckpoint
import pickle
from matplotlib import pyplot
import os
from shutil import copyfile
#from keras import losses
from tensorboard import notebook
import datetime
import time
import natsort

Using TensorFlow backend.


In [3]:
hindi_char= chr(2307)
CATEGORIES= []
for i in range(2305, 2366):
    if not i in (2308, 2317, 2321, 2345, 2353, 2356, 2362, 2363, 2364):
        hindi_char= chr(i)
        CATEGORIES.append(hindi_char)
        
CATEGORIES.append(chr(2381))
CATEGORIES.append(chr(2395))
CATEGORIES.append(chr(2396))
CATEGORIES.append(chr(2397))
CATEGORIES.append(chr(2398))
CATEGORIES.append(chr(2400))
CATEGORIES.append(chr(2401))
CATEGORIES.append(chr(2404))
CATEGORIES.append(chr(2405))
print(len(CATEGORIES))
print(CATEGORIES)
print(CATEGORIES[3])

61
['ँ', 'ं', 'ः', 'अ', 'आ', 'इ', 'ई', 'उ', 'ऊ', 'ऋ', 'ऌ', 'ऎ', 'ए', 'ऐ', 'ऒ', 'ओ', 'औ', 'क', 'ख', 'ग', 'घ', 'ङ', 'च', 'छ', 'ज', 'झ', 'ञ', 'ट', 'ठ', 'ड', 'ढ', 'ण', 'त', 'थ', 'द', 'ध', 'न', 'प', 'फ', 'ब', 'भ', 'म', 'य', 'र', 'ल', 'ळ', 'व', 'श', 'ष', 'स', 'ह', 'ऽ', '्', 'ज़', 'ड़', 'ढ़', 'फ़', 'ॠ', 'ॡ', '।', '॥']
अ


In [ ]:
if not os.path.exists('./images/'):
    os.mkdir('./images/')
for i in CATEGORIES:
    hindi_char= i
    create_path= './images/'+ hindi_char
    if not os.path.exists(create_path):
        os.mkdir(create_path)

In [ ]:
rootdir= '/kaggle/input/bharati-braille-characters/hindiBrailleChar_1/'
for folder in os.listdir(rootdir):
    path_file= rootdir+folder+'/'
    for file in os.listdir(path_file):
        letter= file[0]
        copyfile(path_file+ file, './images/' + letter + '/' + file)
print("success")

In [ ]:
train_dir= './images/'
test_dir= './images/'

train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range= 20,
#         height_shift_range= 0.1,
#         validation_split=0.2,
) # Generator for our training data
validation_image_generator = ImageDataGenerator(rescale=1./255) # Generator for our validation data


train_data_gen = train_datagen.flow_from_directory(batch_size=32,
                                                           directory=train_dir,
                                                           color_mode='grayscale',
                                                           shuffle=True,
                                                           target_size=(28, 28),
                                                           class_mode='binary')

val_data_gen = validation_image_generator.flow_from_directory(batch_size=32,
                                                              directory=test_dir,
                                                              color_mode='grayscale',
                                                              target_size=(28, 28),
                                                              class_mode='binary')
# val_data_predict= flow_from_directory(batch_size=32,
#                                     directory=test_dir,
#                                     color_mode='grayscale',
#                                     target_size=(28, 28),
#                                     class_mode='binary')

In [ ]:
dense_layers= [3]
layer_sizes= [256]
conv_layers= [3]

In [ ]:
for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        for conv_layer in conv_layers:
            NAME= "{}-conv-{}-layers-{}-dense-{}".format(conv_layer, layer_size, dense_layer, int(time.time()))
            print(NAME)
            print("new model training below:\n\n\n")
            
            model = Sequential()

            model.add(Conv2D(layer_size, (3, 3), input_shape=(28,28,1), activation = "relu"))

            model.add(MaxPooling2D(pool_size=(2, 2)))
            
            for l in range(conv_layer-1):
                model.add(Conv2D(layer_size, (3, 3), activation = "relu", padding='same'))
                model.add(MaxPooling2D(pool_size=(2, 2)))
                
            
            model.add(Flatten()) # this converts our 3D feature maps to 1D feature vectors
            for l in range(dense_layer):
                model.add(Dense(layer_size, activation= "relu"))

            
            model.add(Dense(61, activation = "softmax"))

            model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
            filepath="weights.best.hdf5"
            checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
            
            path= "./logs/fit"+ datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
            
            tensorboard_callback = TensorBoard(log_dir= 'logs/{}'.format(NAME))

            history= model.fit_generator(
                train_data_gen,
                steps_per_epoch=138,
                epochs=30,
                validation_data=val_data_gen,
                validation_steps=9,
                callbacks=[checkpoint]
            )
            model.save("model.h5")

In [ ]:
IMG_SIZE=28
model.load_weights('model.h5')
# model = tf.keras.models.load_model('weights.best.hdf5')
acc = model.evaluate_generator(val_data_gen)[1]
print('model accuracy: {}'.format(round(acc,4)))
# model.evaluate()

# model = load_model('model.h5')